# Simple Neural Network implementation on Pytorch

In [ ]:
import pandas as pd
import numpy as np

## Read the data
- Using Pandas `read_csv` functionality to grab the data

In [ ]:
df = pd.read_csv("../input/tabular-playground-series-nov-2021/train.csv", header=0, index_col=0)
df.head(10)

## Separate out features and corresponding target values
- In pandas one can simply provide a `list of columns` as an index to `pd.DataFrame` to get all the data associated to given columns as an index
- `features_cols = ["f0", "f1", ...]`
- `target_cols = "target"`

In [ ]:
# Get the feature and target columns
feature_cols = df.columns[:-1]
target_cols = df.columns[-1]

# Get the data as a numpy matrix
features = df[feature_cols].to_numpy(dtype = np.float32)
target = df[target_cols].to_numpy(dtype = np.float32)
print(f"Data shape: features -> {features.shape}, and Target -> {target.shape}") 

## Standardize the data
- Bring all of the features to `0 mean`, and `standard deviation 1`
- It's requred, other wise the model will be `ralatively over attentive` towards features with `larger scales`.

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# Create the StandardScaler object
scaler = StandardScaler()

# Transform the features
features = scaler.fit_transform(features)

## Split the data into train and validation set
- As we know that there isn't any **validation set** given.
- We need to verify the performance of the model on **unseen dataset**.
- Thus we need to make validation dataset from the given training data.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(features, target, train_size = 0.90, random_state = 42)
print(f"Train data: {X_train.shape}, {Y_train.shape}, \nValidation data: {X_val.shape}, {Y_val.shape}")

## Lets work with pytorch and get our model ready

In [ ]:
import torch
from torch.utils.data import Dataset


## Create a Pytorch Dataset
- Pytorch needs a dataset as a subclass of `torch.utils.data.Dataset`.
- The subclass implements some functions to augument the `Dataset` class for custom datasets.
- They are generally of 2 types 
    1. Iterable-style : Implements the methods `__iter__()` [Useful when we can't read the data randomly]
    2. Map-style : Implements the method `__getitem__()`, and `__len__()` [Heavily used for dataset where we can access data through indexing.]
- In this case `Map-style` dataset subclass is created, to give features, and target values at the query index.

In [ ]:
class TabularDataset(Dataset):
    def __init__(self, x, y, transform = None, target_transform = None):
        self.x = x
        self.y = y
        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(self, idx):
        feature = self.x[idx, :]
        target = self.y[idx]
        target = np.array([target], dtype = np.float32) 

        if self.transform:
            feature = self.transform(feature)
        if self.target_transform:
            target = self.target_transform(target)
        
        return torch.from_numpy(feature), torch.from_numpy(target)
    
    def __len__(self):
        return len(self.x)

In [ ]:
Train_dataset = TabularDataset(X_train, Y_train)
Val_dataset = TabularDataset(X_val, Y_val)

## Make the Dataloaders
- Dataloader modalit is used in `PyTorch` to acces the bulk data with required additional operation on overall datasets like shuffle the dataset.
- Dataloader provides iterable object, to acces the data while training or testing process.

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
# Let's create the dataloader for train and test datasets
train_dataloader = DataLoader(Train_dataset, batch_size = 128*5, shuffle = True)
val_dataloader = DataLoader(Val_dataset, batch_size = 128*5, shuffle = True)

In [ ]:
# Let's see what can be done with these iterators
dataitr = iter(train_dataloader)
features, labels = dataitr.next()
print(f"Features: {features.shape} \nLabels: {labels.shape}")

## Create the Model
- In `PyTorch` the model can be created either in `nn.Sequential` or as a subclass of `nn.Module` with implementation of `forward` function.
- The former is easy to deal with, but the later provides **flexibility**.

In [ ]:
import torch.nn as nn

In [ ]:
# Create Model subclass to define the network
class Model(nn.Module):
    def __init__(self, in_features = 100):
        super().__init__()

        # Define possible layers configuration
        self.fc1 = nn.Linear(in_features, 150)
        self.fc2 = nn.Linear(150, 90)
        self.fc3 = nn.Linear(90, 70)
        self.fc4 = nn.Linear(70, 50)
        self.fc5 = nn.Linear(50, 30)
        self.fc6 = nn.Linear(30, 20)
        self.fc7 = nn.Linear(20, 10)
        self.fc8 = nn.Linear(10, 5)
        self.fc9 = nn.Linear(5, 1)
        
        # Define activations, classifier layer, 
        # and if required then regularizations
        self.activation = nn.SELU() # Activations
        self.classifier = nn.Sigmoid() # Classifier
        self.dropout = nn.Dropout(p=0.1) # Regularization
    
    def forward(self, x):
        """
        Function implements the `forward` pass of a network.
        While training this will run with gradient enabled, to backprop,
        otherwise while testing this is used with torch.no_grad() to infer on the query.
        """
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.activation(self.fc4(x))
        x = self.activation(self.fc5(x))
        x = self.activation(self.fc6(x))
        x = self.activation(self.fc7(x))
        x = self.activation(self.fc8(x))
        x = self.classifier(self.fc9(x))
        
        return x

### We can specify the accelerator device for the model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"{device} device will be used.")

In [ ]:
# Transfer the model parameters and properties to selected device
model = Model().to(torch.device(device))

## Get the model description

In [ ]:
try:
    from torchsummary import summary
except:
    print("Installing Torchsummary..........")
    ! pip install torchsummary
    from torchsummary import summary

In [ ]:
summary(model, (100,))

In [ ]:
def get_acc(acc_type = 'val'):
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        dl = val_dataloader if acc_type == 'val' else train_dataloader
        for data in dl:
            features, labels = data
            features = features.to(device)
            labels = labels.to(device)

            # calculate outputs by running images through the network
            outputs = model(features)

            # the class with the highest energy is what we choose as prediction
            pivot = torch.tensor([0.5]).to(device)
            value = torch.tensor([0.0]).to(device)
            predicted = torch.heaviside(outputs.data-pivot, value)
            
            # print(predicted, labels)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    return 100*correct/total


## Initialize the model weights

In [ ]:
def init_weights(layer):
    if isinstance(layer, nn.Linear):
        nn.init.xavier_normal_(layer.weight.data)

model.apply(init_weights)

## We can also load previously stored model
- set `load_prev` to `True`

In [ ]:
load_prev = False
if load_prev:
    model.load_state_dict(torch.load('./basemodel'))
    print(model.eval())

## Define the optimization algorithm, and loss function

In [ ]:
import torch.optim as optim

criterion = nn.BCELoss() # Loss function
params_list = model.parameters() # model parameters

## We can apply custom learning rate or any other perameters to each layer, use the following:
# params_list = [
#     {'params': model.fc1.parameters(), 'lr': 0.01},
#     {'params': model.fc1_1.parameters(), 'lr': 0.01},
#     {'params': model.fc2.parameters(), 'lr': 0.005},
#     {'params': model.fc2_2.parameters(), 'lr': 0.005},
#     {'params': model.fc3.parameters(), 'lr': 0.001},
#     {'params': model.fc3_3.parameters(), 'lr': 0.001},
#     {'params': model.fc4.parameters(), 'lr': 0.005},
#     {'params': model.fc4_4.parameters(), 'lr': 0.005},
#     {'params': model.fc5.parameters(), 'lr': 0.001},
#     {'params': model.fc5_5.parameters(), 'lr': 0.001},
# ]
optimizer = optim.AdamW(params_list, lr=0.0007, weight_decay=0.01) # Optimizer

## Implement training loop

In [ ]:
for epoch in range(700):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(torch.device(device))
        labels = labels.to(torch.device(device))

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 400 == 399:    # print every 400 mini-batches
            print('[%d, %5d] loss: %.3f, val accuracy: %.3f' %
                  (epoch + 1, i + 1, running_loss / 400, get_acc('val')))
            running_loss = 0.0

print('Finished Training')

## Check validation and train accuracy

In [ ]:
print(' Validation accuracy of the network: %f %%' % (
    get_acc('val')))
print(' Train accuracy of the network: %f %%' % (
    get_acc('train')))

## Save the model parameters for future usage
- This may be relaoded as further continuation of the training process

In [ ]:
torch.save(model.state_dict(), "./basemodel")

In [ ]:
model.state_dict()

## Let's infer on the test set and submit the predictions

### Load test data

In [ ]:
df_test = pd.read_csv("../input/tabular-playground-series-nov-2021/test.csv", header=0, index_col=0)
df_test.head(10)

### Normalize it, and get the pytorch dataset, and dataloader

In [ ]:
features = scaler.transform(np.float32(df_test.values))
test_dataset = TabularDataset(features, np.ones((len(features),)))
test_dataloader = DataLoader(test_dataset, batch_size = 128)

### Infer the results

In [ ]:
result = []
with torch.no_grad():
    for data in test_dataloader:
        features = data[0].to(device)

        # calculate outputs by running images through the network
        outputs = model(features)

        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        result.extend(predicted.cpu().detach().numpy())


### Create results as a pandas DataFrame

In [ ]:
df_result = pd.DataFrame(np.array([df_test.index.tolist(), result]).T, columns = ['id', 'target'])
df_result.head(5)

### Save the dataframe as a csv file

In [ ]:
df_result.to_csv("submission.csv", index=False)